In [12]:
!pip install torch torchvision tqdm torchnet

  Using cached https://files.pythonhosted.org/packages/6c/4b/c38b5144cf167c4f52288517436ccafefe9dc01b8d1c190e18a6b154cd4a/tqdm-4.31.1-py2.py3-none-any.whl
    100% |████████████████████████████████| 1.4MB 3.1MB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/7d/e3/20f3d364d6c8e5d2353c72a67778eb189176f08e873c9900e10c0287b84b/requests-2.21.0-py2.py3-none-any.whl
    100% |████████████████████████████████| 204kB 647kB/s ta 0:00:01
  Using cached https://files.pythonhosted.org/packages/14/2c/cd551d81dbe15200be1cf41cd03869a46fe7226e7450af7a6545bfc474c9/idna-2.8-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/60/75/f692a584e85b7eaba0e03827b3d51f45f571c2e793dd731e598828d380aa/certifi-2019.3.9-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/bc/a9/01ffebfb562e4274b6487b4bb1ddec7ca55ec7510b22e4c51f14098443b8/chardet-3.0.4-py2.py3-none-any.whl
  Stored in directory: /home/cenk/.cache/pip/wheels/e1/03/fb/1c212c2f20905cdf97

In [13]:
import sys
sys.path.insert(0, '../')

In [21]:
import torch
import torch.nn as nn


from wildcat.engine import MultiLabelMAPEngine
from wildcat.models import resnet101_wildcat
from wildcat.voc import Voc2007Classification
from wildcat.engine import MulticlassEngine
from wildcat.mit67 import Mit67
from wildcat.models import resnet101_wildcat


In [22]:
data = '../data/'
image_size = 224
workers = 4
epochs = 20
start_epoch = 0
batch_size = 16
lr = 0.1
lrp = 0.1
momentum = 0.9
weight_decay = 1e-4
print_freq = 0
resume = None
evaluate = 1
k = 1
alpha = 1
maps = 1


In [23]:
def main_voc2007():
	use_gpu = torch.cuda.is_available()

	# define dataset
	train_dataset = Mit67(data, 'train')
	val_dataset = Mit67(data, 'test')
	num_classes = 67

	# load model
	model = resnet101_wildcat(num_classes, pretrained=True, kmax=k, alpha=alpha, num_maps=maps)
	print('classifier', model.classifier)
	print('spatial pooling', model.spatial_pooling)

	# define loss function (criterion)
	criterion = nn.CrossEntropyLoss()

	# define optimizer
	optimizer = torch.optim.SGD(model.get_config_optim(lr, lrp),
	                            lr=lr,
	                            momentum=momentum,
	                            weight_decay=weight_decay)

	state = {'batch_size': batch_size, 'image_size': image_size, 'max_epochs': epochs,
	         'evaluate': evaluate, 'resume': resume}
	state['difficult_examples'] = True
	state['save_model_path'] = '../expes/models/mit67/'

	engine = MulticlassEngine(state)
	engine.learning(model, criterion, train_dataset, val_dataset, optimizer)


if __name__ == '__main__':
	main_voc2007()


[dataset] MIT67 set=train  number of classes=67  number of images=5360
[dataset] MIT67 set=test  number of classes=67  number of images=1340
classifier Sequential(
  (0): Conv2d(2048, 67, kernel_size=(1, 1), stride=(1, 1))
)
spatial pooling Sequential(
  (class_wise): ClassWisePool (num_maps=1)
  (spatial): WildcatPool2d (kmax=1, kmin=1, alpha=1)
)


RuntimeError: CUDA error: out of memory

In [25]:
def main_voc2007():
	use_gpu = torch.cuda.is_available()

	# define dataset
	train_dataset = Voc2007Classification(data, 'trainval')
	val_dataset = Voc2007Classification(data, 'test')
	num_classes = 20

	# load model
	model = resnet101_wildcat(num_classes, pretrained=True, kmax=k, alpha=alpha, num_maps=maps)
	print('classifier', model.classifier)
	print('spatial pooling', model.spatial_pooling)

	# define loss function (criterion)
	criterion = nn.MultiLabelSoftMarginLoss()

	# define optimizer
	optimizer = torch.optim.SGD(model.get_config_optim(lr, lrp),
	                            lr=lr,
	                            momentum=momentum,
	                            weight_decay=weight_decay)

	state = {'batch_size': batch_size, 'image_size': image_size, 'max_epochs': epochs,
	         'evaluate': evaluate, 'resume': resume}
	state['difficult_examples'] = True
	state['save_model_path'] = '../expes/models/voc2007/'

	engine = MultiLabelMAPEngine(state)
	engine.learning(model, criterion, train_dataset, val_dataset, optimizer)


if __name__ == '__main__':
	main_voc2007()

VOCdevkit_18-May-2011.tar: 0.00B [00:00, ?B/s]

Downloading: "http://host.robots.ox.ac.uk/pascal/VOC/voc2012/VOCdevkit_18-May-2011.tar" to ../data/tmp/VOCdevkit_18-May-2011.tar



VOCdevkit_18-May-2011.tar: 516kB [05:07, 1.68kB/s]                             
VOCtrainval_06-Nov-2007.tar: 0.00B [00:00, ?B/s]

[dataset] Extracting tar file ../data/tmp/VOCdevkit_18-May-2011.tar to ../data/
[dataset] Done!
Downloading: "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar" to ../data/tmp/VOCtrainval_06-Nov-2007.tar



VOCtrainval_06-Nov-2007.tar: 460MB [09:33, 803kB/s]                                 


[dataset] Extracting tar file ../data/tmp/VOCtrainval_06-Nov-2007.tar to ../data/


VOCtest_06-Nov-2007.tar: 0.00B [00:00, ?B/s]

[dataset] Done!
Downloading: "http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar" to ../data/tmp/VOCtest_06-Nov-2007.tar



ConnectionResetError: [Errno 104] Connection reset by peer